# Modifications

Link to Interactive Notebook: 
https://colab.research.google.com/drive/1sbJTsgCsAQwCkGdLXK7EdgkaTpiTGBM1#scrollTo=T71qxHyh9p23

1. Reduced Input Dimension
2. Different handling of xco2: merge the value in as an additional input in the fully connected layer (?)

In [1]:
import imp
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
import sys
import tensorflow as tf
from tensorflow import keras
import os
from sklearn.model_selection import train_test_split

#!module load cuda11.0/toolkit cuda11.0/blas cudnn8.0-cuda11.0

#print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

#tf.config.list_physical_devices()

sys.path.insert(0, '../src')

from utils import df_to_xarray,read_xarray,plot_image

/burg/glab/users/sk4973/venv/lib/python3.8/site-packages/xarray/backends/cfgrib_.py:27: UserWarning: Failed to load cfgrib - most likely there is a problem accessing the ecCodes library. Try `import cfgrib` to get the full error message
  warnings.warn(
2021-09-22 16:29:45.504520: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


In [2]:
# Reading Data
dir_name="../data/data1"
val_dir_name="../data/data2"


chl,mld,sss,sst,u10,fg_co2,xco2,icefrac,patm,pco2=read_xarray(dir_name)



/burg/glab/users/sk4973/venv/lib/python3.8/site-packages/xarray/backends/plugins.py:61: RuntimeWarning: Engine 'cfgrib' loading failed:
Cannot find the ecCodes library
  warnings.warn(f"Engine {name!r} loading failed:\n{ex}", RuntimeWarning)


In [12]:
def convert_nan(arr):
    nans=np.isnan(arr)
    min_val=arr[~nans].min()
    #print(min_val)
    arr[nans]=min_val-1
    return arr

def add_dimension(arr):
    images=np.expand_dims(arr, axis=3)
    return images

def scale_image(arr):
    ## Normal
    #arr=(arr-np.mean(arr))/np.std(arr)
    
    ## Min-Max
    # min_val=arr.min()
    # max_val=arr.max()
    # arr=arr/(min_val-max_val)

    ## Image Scale
    min_pixel = arr.min() 
    max_pixel = arr.max()
    new_min = 0
    new_max = 255
    arr = (arr-min_pixel)*(255)/(max_pixel-min_pixel)+new_min 
    return arr
  

def preprocess_image_reduced(data,xco2=False):
  """
  dimension reduced the output should be  (180,360,5)
  """
  if xco2:
    return data
  return scale_image(convert_nan(data))

In [20]:
#xco2 values are a constant value across the globe, so creating an image layer with constant value for the model
xco2_images=[]
min_xco2=np.min(xco2.XCO2.data)
max_xco2=np.max(xco2.XCO2.data)
new_min=0
new_max=255
print(min_xco2, max_xco2)
print()


for i in xco2.XCO2.data:
    num = (i-min_xco2)*(new_max-new_min)/(max_xco2-min_xco2)+new_min
    tmp = (np.repeat(num,180*360)).reshape(180,-1)
    xco2_images.append(tmp)

xco2_images=np.array(xco2_images)

print("xco2 shape: ")
print(xco2_images.shape, chl.Chl.data.shape)



chl_images=preprocess_image_reduced(chl.Chl.data)
mld_images=preprocess_image_reduced(mld.MLD.data)
sss_images=preprocess_image_reduced(sss.SSS.data)
sst_images=preprocess_image_reduced(sst.SST.data)
xco2_images=preprocess_image_reduced(xco2_images,xco2=True)
pco2_images=preprocess_image_reduced(pco2.pCO2.data)

X = np.stack((chl_images, mld_images, sss_images, sst_images,xco2_images), axis = 1)


340.84854 407.2084

xco2 shape: 
(421, 180, 360) (421, 180, 360)


In [21]:
X=X.reshape((421,180,360,5))
X.shape

(421, 180, 360, 5)

In [8]:
INPUT_SHAPE=X[0].shape
OUTPUT_SHAPE=pco2_images[0].shape

In [27]:
from functools import partial

DefaultConv2D = partial(keras.layers.Conv2D,
                        kernel_size=5,activation='relu', padding="SAME")


base_model = keras.models.Sequential([
    DefaultConv2D(filters=64, input_shape=INPUT_SHAPE),
    DefaultConv2D(filters=64),
    keras.layers.MaxPooling2D(pool_size=3),
    keras.layers.Dropout(0.3),

    DefaultConv2D(filters=128),
    DefaultConv2D(filters=128),

    keras.layers.UpSampling2D(size=3),
    DefaultConv2D(filters=64),    
    DefaultConv2D(filters=2),
    DefaultConv2D(filters=1,kernel_size=1),
    keras.layers.Reshape(OUTPUT_SHAPE)
   
])

base_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 180, 360, 64)      8064      
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 180, 360, 64)      102464    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 60, 120, 64)       0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 60, 120, 64)       0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 60, 120, 128)      204928    
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 60, 120, 128)      409728    
_________________________________________________________________
up_sampling2d_1 (UpSampling2 (None, 180, 360, 128)    

In [28]:
base_model.compile(loss="mean_squared_error", optimizer="nadam", metrics=["mean_squared_error"])

In [29]:
model_path="../models/base_model/base_model_new.h5"
early_stopings = tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=1, mode='min')
checkpoint =  tf.keras.callbacks.ModelCheckpoint(model_path, monitor='val_loss', save_best_only=True, mode='min', verbose=0)
callbacks=[early_stopings,checkpoint]

history = base_model.fit(X,pco2_images, epochs=100, validation_data=(X,pco2_images),workers=-1,batch_size=32,callbacks=callbacks)



Epoch 1/100
14/14 [==============================] - 15s 558ms/step - loss: 1988068.3558 - mean_squared_error: 1988068.6517 - val_loss: 4246.2490 - val_mean_squared_error: 4246.2490
Epoch 2/100
14/14 [==============================] - 5s 387ms/step - loss: 4320.3524 - mean_squared_error: 4320.3524 - val_loss: 4548.3579 - val_mean_squared_error: 4548.3584
Epoch 3/100
14/14 [==============================] - 5s 387ms/step - loss: 3812.2001 - mean_squared_error: 3812.2002 - val_loss: 4216.2769 - val_mean_squared_error: 4216.2769
Epoch 4/100
14/14 [==============================] - 5s 387ms/step - loss: 3703.0420 - mean_squared_error: 3703.0423 - val_loss: 4138.6978 - val_mean_squared_error: 4138.6978
Epoch 5/100
14/14 [==============================] - 5s 389ms/step - loss: 3787.9596 - mean_squared_error: 3787.9596 - val_loss: 3763.2827 - val_mean_squared_error: 3763.2827
Epoch 6/100
14/14 [==============================] - 5s 388ms/step - loss: 3641.6108 - mean_squared_error: 3641.6108 -

In [ ]:
predicted_image=base_model.predict(X[419:421],verbose=1)

In [ ]:
plot_image(np.squeeze(predicted_image[1]))

In [ ]:
difference=np.squeeze(pco2_images[419:421][1])-np.squeeze(predicted_image[0])
plot_image(difference)

In [ ]:
plot_image(np.squeeze(pco2_images[419:421][1]))

In [ ]:
model1 = keras.models.Sequential([
    DefaultConv2D(filters=64, input_shape=INPUT_SHAPE),
    DefaultConv2D(filters=64),
    keras.layers.MaxPooling2D(pool_size=2),
    keras.layers.Dropout(0.3),

    DefaultConv2D(filters=128),
    DefaultConv2D(filters=128),
    keras.layers.MaxPooling2D(pool_size=2),
    keras.layers.Dropout(0.3),

    DefaultConv2D(filters=256),
    DefaultConv2D(filters=256),
    keras.layers.Dropout(0.3),

    keras.layers.UpSampling2D(size=2),
    DefaultConv2D(filters=128),
    DefaultConv2D(filters=128),

    keras.layers.UpSampling2D(size=2),
    DefaultConv2D(filters=64),
    DefaultConv2D(filters=64),
    
    DefaultConv2D(filters=2),
    DefaultConv2D(filters=2),
    DefaultConv2D(filters=1,kernel_size=1),
    keras.layers.Reshape(OUTPUT_SHAPE)
   
])

model1.summary()

In [ ]:
model1.compile(loss="mean_squared_error", optimizer="nadam", metrics=["mean_squared_error"])